## CEPARCO CUDA Project Group 1

In [1]:
import os

# Add the directory containing the executable to the PATH
os.environ["PATH"] += os.pathsep + "/usr/local/cuda/bin"

# Check if the directory is added to the PATH
print(os.environ["PATH"])

/opt/tljh/user/bin:/bin:/usr/bin:/usr/local/cuda/bin


# C Program

In [4]:
%%writefile C_asum.c

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <math.h> //fabsf and cos/sin functions

void asumfunc(size_t n, double* a, double* asum) {
  *asum = 0.0;
  for (int i=0; i<n;i++)
     *asum += fabs(a[i]);
}

int main(int argc, char** argv){
   const size_t N = 28;
   const size_t ARRAY_SIZE = 1<<N;
   const size_t ARRAY_BYTES = ARRAY_SIZE * sizeof(double);
   const size_t loope = 30;

   double *a, *asum;
   a = (double*)malloc(ARRAY_BYTES);
   asum = (double*)malloc(sizeof(double));

   clock_t start, end;

   for (int i = 0; i < ARRAY_SIZE; i++) {
    a[i] = sin((double)i * 0.0003) * cos((double)i * 0.0007) * 1000.0;
   }

   *asum = 0.0;

   asumfunc(ARRAY_SIZE,a,asum);

   double elapse, time_taken;
   elapse = 0.0f;
   for (int i=0; i<loope; i++){
    start = clock();
     asumfunc(ARRAY_SIZE,a,asum);
    end = clock();
    time_taken = ((double)(end-start))*1E3/CLOCKS_PER_SEC;
    elapse = elapse + time_taken;
   }

    printf("Function (in C) average time for %lu loops is %f milliseconds to execute an array size %lu \n", loope, elapse/loope, ARRAY_SIZE);
    printf("Absolute sum of vector size 2^%lu: %lf \n",N,*asum);


  double err_asum = 0.0;
   for (int i=0; i<ARRAY_SIZE; i++)
        err_asum += fabs(a[i]);
   if (fabs(err_asum - *asum) > 1e-2)
        printf("Error encountered: \n Function result: %lf \n Error checking result: %lf \n Error difference: %lf \n", *asum, err_asum, (err_asum - *asum));
   else
        printf("No errors encountered. \n (Difference less than 1e-2) \n Function result: %lf \n Error checking result: %lf \n Error difference: %lf \n", *asum, err_asum, (err_asum - *asum));

    free(a);
    return 0;
}

Overwriting C_asum.c


In [5]:
%%bash
gcc C_asum.c -lm -o C_asum

In [6]:
%%bash
./C_asum

Function (in C) average time for 30 loops is 2421.755667 milliseconds to execute an array size 268435456 
Absolute sum of vector size 2^28: 108762865473.985641 
No errors encountered. 
 (Difference less than 1e-2) 
 Function result: 108762865473.985641 
 Error checking result: 108762865473.985641 
 Error difference: 0.000000 


# Grid Stride; no prefetch, no page creation, no mem advise

In [7]:
%%writefile CUDA_asum1.cu

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <math.h> //fabsf and cos/sin functions

__global__
void asumfunc(size_t n, double* a, double *asum) {
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    for (int i = index; i < n; i += stride)
        atomicAdd(asum,fabs(a[i])); //proper summation function for CUDA
}

int main(){
   const size_t N = 28;
   const size_t ARRAY_SIZE = 1<<N;
   const size_t ARRAY_BYTES = ARRAY_SIZE * sizeof(double);
   const size_t loope = 30;

   double *a, *asum;
   cudaMallocManaged(&a, ARRAY_BYTES);
   cudaMallocManaged(&asum, sizeof(double));

   for (int i = 0; i < ARRAY_SIZE; i++) {
    a[i] = sin((double)i * 0.0003) * cos((double)i * 0.0007) * 1000.0;
   }

   *asum = 0.0;

  size_t numThreads = 1024;
  size_t numBlocks = (ARRAY_SIZE + numThreads-1) / numThreads;

  printf("*** function = Double asum\n");
  printf("numElements = %lu\n", ARRAY_SIZE);
  printf("numBlocks = %lu, numThreads = %lu \n",numBlocks,numThreads);
  for (size_t i=0; i<loope;i++){
    *asum = 0.0;
    asumfunc <<<numBlocks, numThreads>>> (ARRAY_SIZE,a,asum);
    cudaDeviceSynchronize();
  } 
  //Because the result makes use of a summation function, it is different
  //From the CUDA square function where it does not overwrite another array,
  //hence each kernel must not overlap their results over another. To fix this,
  //cudaDeviceSynchronize() was placed within the loop, as without it *asum = 0.0
  //is not properly set and causes the printed result to appear loope times larger.

  printf("Absolute sum of vector size 2^%lu: %lf \n",N,*asum);

  //Since summation is performed by each kernel, there is no "element by element" error checking
  double err_asum = 0.0;
   for (int i=0; i<ARRAY_SIZE; i++)
        err_asum += fabs(a[i]);
   if (fabs(err_asum - *asum) > 1e-2)
        printf("Error encountered: \n Function result: %lf \n Error checking result: %lf \n Error difference: %lf \n", *asum, err_asum, (err_asum - *asum));
   else
        printf("No errors encountered. \n (Difference less than 1e-2) \n Function result: %lf \n Error checking result: %lf \n Error difference: %lf \n", *asum, err_asum, (err_asum - *asum));

  cudaFree(a);
  cudaFree(asum);
  return 0;
}
//In compiling, -arch=sm_60 allows CUDA 6.0 for use of atomicAdd with double* and double parameters

Overwriting CUDA_asum1.cu


In [8]:
%%bash
nvcc CUDA_asum1.cu -lm -o CUDA_asum1 -Wno-deprecated-gpu-targets -arch=sm_60

In [9]:
%%bash
nvprof ./CUDA_asum1

==1008037== NVPROF is profiling process 1008037, command: ./CUDA_asum1


*** function = Double asum
numElements = 268435456
numBlocks = 262144, numThreads = 1024 
Absolute sum of vector size 2^28: 108762865473.981369 
No errors encountered. 
 (Difference less than 1e-2) 
 Function result: 108762865473.981369 
 Error checking result: 108762865473.985641 
 Error difference: 0.004272 


==1008037== Profiling application: ./CUDA_asum1
==1008037== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  20.1592s        30  671.97ms  614.79ms  1.98873s  asumfunc(unsigned long, double*, double*)
      API calls:   89.87%  20.1620s        30  672.07ms  614.92ms  1.98889s  cudaDeviceSynchronize
                    9.07%  2.03515s         2  1.01758s  1.1067ms  2.03405s  cudaMallocManaged
                    0.67%  151.41ms         2  75.703ms  698.66us  150.71ms  cudaFree
                    0.36%  81.105ms        30  2.7035ms  259.39us  67.165ms  cudaLaunchKernel
                    0.02%  4.1609ms         1  4.1609ms  4.1609ms  4.1609ms  cuDeviceTotalMem
                    0.00%  863.91us       114  7.5780us     106ns  404.52us  cuDeviceGetAttribute
                    0.00%  251.68us         1  251.68us  251.68us  251.68us  cuDeviceGetName
                    0.00%  44.340us         1  44.340us  44.34

# Grid Stride; with prefetch, no page creation, no mem advise

In [10]:
%%writefile CUDA_asum2.cu

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>

__global__
void asumfunc(size_t n, double* a, double *asum) {
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    for (int i = index; i < n; i += stride)
        atomicAdd(asum,fabs(a[i]));
}


int main(){
   const size_t N = 28;
   const size_t ARRAY_SIZE = 1<<N;
   const size_t ARRAY_BYTES = ARRAY_SIZE * sizeof(double);
   const size_t loope = 30;

   double *a, *asum;
   cudaMallocManaged(&a, ARRAY_BYTES);
   cudaMallocManaged(&asum, sizeof(double));

  int device = -1;
  cudaGetDevice(&device);

   for (int i = 0; i < ARRAY_SIZE; i++) {
    a[i] = sin((double)i * 0.0003) * cos((double)i * 0.0007) * 1000.0;
   }

   *asum = 0.0;

  cudaMemPrefetchAsync(a,ARRAY_BYTES,device,NULL);

  size_t numThreads = 1024;
  size_t numBlocks = (ARRAY_SIZE + numThreads-1) / numThreads;

  printf("*** function = Double asum\n");
  printf("numElements = %lu\n", ARRAY_SIZE);
  printf("numBlocks = %lu, numThreads = %lu \n",numBlocks,numThreads);
  for (size_t i=0; i<loope;i++){
    *asum = 0.0;
    asumfunc <<<numBlocks, numThreads>>> (ARRAY_SIZE,a,asum);
    cudaDeviceSynchronize();
  }

//"Prefetch data" from GPU-CPU
  cudaMemPrefetchAsync(a,ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(asum,sizeof(double),cudaCpuDeviceId,NULL);

  printf("Absolute sum of vector size 2^%lu: %lf \n",N,*asum);

  double err_asum = 0.0;
   for (int i=0; i<ARRAY_SIZE; i++)
        err_asum += fabs(a[i]);
   if (fabs(err_asum - *asum) > 1e-2)
        printf("Error encountered: \n Function result: %lf \n Error checking result: %lf \n Error difference: %lf \n", *asum, err_asum, (err_asum - *asum));
   else
        printf("No errors encountered. \n (Difference less than 1e-2) \n Function result: %lf \n Error checking result: %lf \n Error difference: %lf \n", *asum, err_asum, (err_asum - *asum));

  cudaFree(a);
  cudaFree(asum);
  return 0;
}

Overwriting CUDA_asum2.cu


In [11]:
%%bash
nvcc CUDA_asum2.cu -lm -o CUDA_asum2 -Wno-deprecated-gpu-targets -arch=sm_60

In [12]:
%%bash
nvprof ./CUDA_asum2

==1008576== NVPROF is profiling process 1008576, command: ./CUDA_asum2


*** function = Double asum
numElements = 268435456
numBlocks = 262144, numThreads = 1024 
Absolute sum of vector size 2^28: 108762865473.984619 
No errors encountered. 
 (Difference less than 1e-2) 
 Function result: 108762865473.984619 
 Error checking result: 108762865473.985641 
 Error difference: 0.001022 


==1008576== Profiling application: ./CUDA_asum2
==1008576== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  18.4628s        30  615.43ms  614.75ms  620.26ms  asumfunc(unsigned long, double*, double*)
      API calls:   86.97%  18.4658s        30  615.53ms  614.74ms  620.28ms  cudaDeviceSynchronize
                    6.70%  1.42369s         2  711.84ms  1.0738ms  1.42261s  cudaMallocManaged
                    5.05%  1.07144s         3  357.15ms  1.0313ms  848.08ms  cudaMemPrefetchAsync
                    0.78%  165.36ms         2  82.678ms  1.1198ms  164.24ms  cudaFree
                    0.50%  105.93ms        30  3.5311ms  258.33us  92.310ms  cudaLaunchKernel
                    0.00%  680.75us       114  5.9710us     140ns  297.30us  cuDeviceGetAttribute
                    0.00%  367.64us         1  367.64us  367.64us  367.64us  cuDeviceGetName
                    0.00%  46.289us         1  46.289us  4

# Grid Stride; with prefetch; with page creation; no mem advise

In [13]:
%%writefile CUDA_asum3.cu

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>

__global__
void asumfunc(size_t n, double* a, double *asum) {
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    for (int i = index; i < n; i += stride)
        atomicAdd(asum,fabs(a[i]));
}


int main(){
   const size_t N = 28;
   const size_t ARRAY_SIZE = 1<<N;
   const size_t ARRAY_BYTES = ARRAY_SIZE * sizeof(double);
   const size_t loope = 30;

   double *a, *asum;
   cudaMallocManaged(&a, ARRAY_BYTES);
   cudaMallocManaged(&asum, sizeof(double));

  int device = -1;
  cudaGetDevice(&device);

  cudaMemPrefetchAsync(a,ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(asum,sizeof(double),device,NULL);

   for (int i = 0; i < ARRAY_SIZE; i++) {
    a[i] = sin((double)i * 0.0003) * cos((double)i * 0.0007) * 1000.0;
   }

   *asum = 0.0;

  cudaMemPrefetchAsync(a,ARRAY_BYTES,device,NULL);

  size_t numThreads = 1024;
  size_t numBlocks = (ARRAY_SIZE + numThreads-1) / numThreads;

  printf("*** function = Double asum\n");
  printf("numElements = %lu\n", ARRAY_SIZE);
  printf("numBlocks = %lu, numThreads = %lu \n",numBlocks,numThreads);
  for (size_t i=0; i<loope;i++){
    *asum = 0.0;
    asumfunc <<<numBlocks, numThreads>>> (ARRAY_SIZE,a,asum);
    cudaDeviceSynchronize();
  }

//"Prefetch data" from GPU-CPU
  cudaMemPrefetchAsync(a,ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(asum,sizeof(double),cudaCpuDeviceId,NULL);

  printf("Absolute sum of vector size 2^%lu: %lf \n",N,*asum);

  double err_asum = 0.0;
   for (int i=0; i<ARRAY_SIZE; i++)
        err_asum += fabs(a[i]);
   if (fabs(err_asum - *asum) > 1e-2)
        printf("Error encountered: \n Function result: %lf \n Error checking result: %lf \n Error difference: %lf \n", *asum, err_asum, (err_asum - *asum));
   else
        printf("No errors encountered. \n (Difference less than 1e-2) \n Function result: %lf \n Error checking result: %lf \n Error difference: %lf \n", *asum, err_asum, (err_asum - *asum));

  cudaFree(a);
  cudaFree(asum);
  return 0;
}

Overwriting CUDA_asum3.cu


In [14]:
%%bash
nvcc CUDA_asum3.cu -lm -o CUDA_asum3 -Wno-deprecated-gpu-targets -arch=sm_60

In [15]:
%%bash
nvprof ./CUDA_asum3

==1008802== NVPROF is profiling process 1008802, command: ./CUDA_asum3


*** function = Double asum
numElements = 268435456
numBlocks = 262144, numThreads = 1024 
Absolute sum of vector size 2^28: 108762865473.983490 
No errors encountered. 
 (Difference less than 1e-2) 
 Function result: 108762865473.983490 
 Error checking result: 108762865473.985641 
 Error difference: 0.002151 


==1008802== Profiling application: ./CUDA_asum3
==1008802== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  18.4575s        30  615.25ms  614.57ms  616.13ms  asumfunc(unsigned long, double*, double*)
      API calls:   80.28%  18.4593s        30  615.31ms  614.68ms  616.18ms  cudaDeviceSynchronize
                   12.18%  2.80118s         5  560.24ms  760.33us  1.76196s  cudaMemPrefetchAsync
                    6.05%  1.39158s         2  695.79ms  1.2224ms  1.39036s  cudaMallocManaged
                    0.79%  182.80ms        30  6.0933ms  153.60us  172.27ms  cudaLaunchKernel
                    0.69%  158.36ms         2  79.179ms  1.7866ms  156.57ms  cudaFree
                    0.00%  638.15us       114  5.5970us     110ns  315.59us  cuDeviceGetAttribute
                    0.00%  183.88us         1  183.88us  183.88us  183.88us  cuDeviceGetName
                    0.00%  29.022us         1  29.022us  2

# Grid Stride; with prefetch; with page creation; with mem advise

In [16]:
%%writefile CUDA_asum4.cu

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>

__global__
void asumfunc(size_t n, double* a, double *asum) {
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    for (int i = index; i < n; i += stride)
        atomicAdd(asum,fabs(a[i]));
}


int main(){
   const size_t N = 28;
   const size_t ARRAY_SIZE = 1<<N;
   const size_t ARRAY_BYTES = ARRAY_SIZE * sizeof(double);
   const size_t loope = 30;

   double *a, *asum;
   cudaMallocManaged(&a, ARRAY_BYTES);
   cudaMallocManaged(&asum, sizeof(double));

  int device = -1;
  cudaGetDevice(&device);


// memory advise
   cudaMemAdvise(a, ARRAY_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
   cudaMemAdvise(a, ARRAY_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);

//"prefetch data" to create CPU page memory
  cudaMemPrefetchAsync(a,ARRAY_BYTES,cudaCpuDeviceId,NULL);
//"prefetch data" to create GPU page memory
  cudaMemPrefetchAsync(asum,sizeof(double),device,NULL);

// ****init array
   for (int i = 0; i < ARRAY_SIZE; i++) {
    a[i] = sin((double)i * 0.0003) * cos((double)i * 0.0007) * 1000.0;
   }

   *asum = 0.0;

 //"Prefetch data" from CPU-GPU
  cudaMemPrefetchAsync(a,ARRAY_BYTES,device,NULL);

// setup CUDA kernel
  size_t numThreads = 1024;
  size_t numBlocks = (ARRAY_SIZE + numThreads-1) / numThreads;

  printf("*** function = Double asum\n");
  printf("numElements = %lu\n", ARRAY_SIZE);
  printf("numBlocks = %lu, numThreads = %lu \n",numBlocks,numThreads);
  for (size_t i=0; i<loope;i++){
    *asum = 0.0;
    asumfunc <<<numBlocks, numThreads>>> (ARRAY_SIZE,a,asum);
    cudaDeviceSynchronize();
  }

//"Prefetch data" from GPU-CPU
  cudaMemPrefetchAsync(a,ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(asum,sizeof(double),cudaCpuDeviceId,NULL);

  printf("Absolute sum of vector size 2^%lu: %lf \n",N,*asum);

  double err_asum = 0.0;
   for (int i=0; i<ARRAY_SIZE; i++)
        err_asum += fabs(a[i]);
   if (fabs(err_asum - *asum) > 1e-2)
        printf("Error encountered: \n Function result: %lf \n Error checking result: %lf \n Error difference: %lf \n", *asum, err_asum, (err_asum - *asum));
   else
        printf("No errors encountered. \n (Difference less than 1e-2) \n Function result: %lf \n Error checking result: %lf \n Error difference: %lf \n", *asum, err_asum, (err_asum - *asum));

  cudaFree(a);
  cudaFree(asum);
  return 0;
}

Overwriting CUDA_asum4.cu


In [17]:
%%bash
nvcc CUDA_asum4.cu -lm -o CUDA_asum4 -Wno-deprecated-gpu-targets -arch=sm_60

In [18]:
%%bash
nvprof ./CUDA_asum4

==1008877== NVPROF is profiling process 1008877, command: ./CUDA_asum4


*** function = Double asum
numElements = 268435456
numBlocks = 262144, numThreads = 1024 
Absolute sum of vector size 2^28: 108762865473.985596 
No errors encountered. 
 (Difference less than 1e-2) 
 Function result: 108762865473.985596 
 Error checking result: 108762865473.985641 
 Error difference: 0.000046 


==1008877== Profiling application: ./CUDA_asum4
==1008877== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  18.4556s        30  615.19ms  614.61ms  616.76ms  asumfunc(unsigned long, double*, double*)
      API calls:   83.16%  18.4579s        30  615.26ms  614.69ms  616.82ms  cudaDeviceSynchronize
                    9.72%  2.15840s         5  431.68ms  411.37us  1.83960s  cudaMemPrefetchAsync
                    6.24%  1.38593s         2  692.97ms  695.74us  1.38524s  cudaMallocManaged
                    0.80%  178.37ms         2  89.183ms  9.9370ms  168.43ms  cudaFree
                    0.06%  12.965ms        30  432.15us  196.07us  2.1501ms  cudaLaunchKernel
                    0.00%  449.89us       114  3.9460us     129ns  173.92us  cuDeviceGetAttribute
                    0.00%  214.44us         1  214.44us  214.44us  214.44us  cuDeviceGetName
                    0.00%  134.74us         2  67.369us  1

# Classic MemCopy (no Unified Memory)

In [21]:
%%writefile CUDA_asum5.cu
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <time.h>

__global__
void asumfunc(size_t n, double* a, double *asum) {
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    double localSum = 0.0;

    // Each thread sums part of the array
    for (int i = index; i < n; i += stride)
        localSum += fabs(a[i]);

    // Atomic add partial results to global sum
    atomicAdd(asum, localSum);
}

int main() {
    const size_t N = 28;
    const size_t ARRAY_SIZE = 1 << N;
    const size_t ARRAY_BYTES = ARRAY_SIZE * sizeof(double);
    const size_t loope = 30;

    // Allocate host memory (regular malloc)
    double *h_a = (double*)malloc(ARRAY_BYTES);
    double h_asum = 0.0;

    // Initialize host data
    for (size_t i = 0; i < ARRAY_SIZE; i++)
        h_a[i] = sin((double)i * 0.0003) * cos((double)i * 0.0007) * 1000.0;

    // Allocate device memory (no Unified Memory)
    double *d_a, *d_asum;
    cudaMalloc(&d_a, ARRAY_BYTES);
    cudaMalloc(&d_asum, sizeof(double));

    // Copy data to device
    cudaMemcpy(d_a, h_a, ARRAY_BYTES, cudaMemcpyHostToDevice);
    cudaMemcpy(d_asum, &h_asum, sizeof(double), cudaMemcpyHostToDevice);

    // Kernel setup
    size_t numThreads = 1024;
    size_t numBlocks = (ARRAY_SIZE + numThreads - 1) / numThreads;

    printf("*** function = Double asum (Classic MemCopy)\n");
    printf("numElements = %lu\n", ARRAY_SIZE);
    printf("numBlocks = %lu, numThreads = %lu\n", numBlocks, numThreads);

    // Run kernel multiple times
    double zero = 0.0;
    for (size_t i = 0; i < loope; i++) {
        cudaMemcpy(d_asum, &zero, sizeof(double), cudaMemcpyHostToDevice);
        asumfunc<<<numBlocks, numThreads>>>(ARRAY_SIZE, d_a, d_asum);
        cudaDeviceSynchronize();
    }

    // Copy result back to host
    cudaMemcpy(&h_asum, d_asum, sizeof(double), cudaMemcpyDeviceToHost);

    printf("Absolute sum of vector size 2^%lu: %lf\n", N, h_asum);

    // Validate result
    double err_asum = 0.0;
    for (size_t i = 0; i < ARRAY_SIZE; i++)
        err_asum += fabs(h_a[i]);

    if (fabs(err_asum - h_asum) > 1e-2)
        printf("Error encountered: \n Function result: %lf \n Error checking result: %lf \n Error difference: %lf \n", h_asum, err_asum, (err_asum - h_asum));
    else
        printf("No errors encountered. \n (Difference less than 1e-2) \n Function result: %lf \n Error checking result: %lf \n Error difference: %lf \n", h_asum, err_asum, (err_asum - h_asum));

    // Free memory
    cudaFree(d_a);
    cudaFree(d_asum);
    free(h_a);

    return 0;
}

Overwriting CUDA_asum5.cu


In [22]:
%%bash
nvcc CUDA_asum5.cu -lm -o CUDA_asum5 -Wno-deprecated-gpu-targets -arch=sm_60

In [23]:
%%bash
nvprof ./CUDA_asum5

==1008993== NVPROF is profiling process 1008993, command: ./CUDA_asum5


*** function = Double asum (Classic MemCopy)
numElements = 268435456
numBlocks = 262144, numThreads = 1024
Absolute sum of vector size 2^28: 108762865473.984940
No errors encountered. 
 (Difference less than 1e-2) 
 Function result: 108762865473.984940 
 Error checking result: 108762865473.985641 
 Error difference: 0.000702 


==1008993== Profiling application: ./CUDA_asum5
==1008993== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   89.37%  18.4266s        30  614.22ms  613.67ms  627.89ms  asumfunc(unsigned long, double*, double*)
                   10.63%  2.19124s        32  68.476ms     640ns  2.19122s  [CUDA memcpy HtoD]
                    0.00%  3.4880us         1  3.4880us  3.4880us  3.4880us  [CUDA memcpy DtoH]
      API calls:   83.95%  18.4294s        30  614.31ms  613.74ms  627.96ms  cudaDeviceSynchronize
                   10.02%  2.20024s        33  66.674ms  47.516us  2.19254s  cudaMemcpy
                    5.77%  1.26690s         2  633.45ms  329.57us  1.26657s  cudaMalloc
                    0.23%  49.851ms        30  1.6617ms  121.21us  43.886ms  cudaLaunchKernel
                    0.03%  5.8133ms         2  2.9066ms  1.0135ms  4.7998ms  cudaFree
                    0.00%  641.33us       114  5.6250us     106ns  251.94u

# Grid-Stride Loop with Prefetch and GPU Data Initialization

In [24]:
%%writefile CUDA_asum6.cu
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>

__global__
void init_array(size_t n, double* a) {
    size_t index = blockIdx.x * blockDim.x + threadIdx.x;
    size_t stride = blockDim.x * gridDim.x;
    for (size_t i = index; i < n; i += stride) {
        a[i] = sin((double)i * 0.0003) * cos((double)i * 0.0007) * 1000.0;
    }
}

__global__
void asumfunc(size_t n, double* a, double *asum) {
    size_t index = blockIdx.x * blockDim.x + threadIdx.x;
    size_t stride = blockDim.x * gridDim.x;
    for (size_t i = index; i < n; i += stride)
        atomicAdd(asum, fabs(a[i]));
}

int main() {
    const size_t N = 28;
    const size_t ARRAY_SIZE = 1 << N;
    const size_t ARRAY_BYTES = ARRAY_SIZE * sizeof(double);
    const size_t loope = 30;

    double *a, *asum;
    cudaMallocManaged(&a, ARRAY_BYTES);
    cudaMallocManaged(&asum, sizeof(double));

    int device = -1;
    cudaGetDevice(&device);

    // Prefetch to GPU before initializing
    cudaMemPrefetchAsync(a, ARRAY_BYTES, device, NULL);
    cudaMemPrefetchAsync(asum, sizeof(double), device, NULL);

    size_t numThreads = 1024;
    size_t numBlocks = (ARRAY_SIZE + numThreads - 1) / numThreads;

    // GPU kernel to initialize array
    init_array<<<numBlocks, numThreads>>>(ARRAY_SIZE, a);
    cudaDeviceSynchronize();

    *asum = 0.0;

    printf("*** function = Double asum (GPU init data)\n");
    printf("numElements = %lu\n", ARRAY_SIZE);
    printf("numBlocks = %lu, numThreads = %lu \n", numBlocks, numThreads);

    for (size_t i=0; i<loope;i++){
      *asum = 0.0;
      asumfunc <<<numBlocks, numThreads>>> (ARRAY_SIZE,a,asum);
      cudaDeviceSynchronize();
    }

    // Prefetch back to CPU for validation
    cudaMemPrefetchAsync(a, ARRAY_BYTES, cudaCpuDeviceId, NULL);
    cudaMemPrefetchAsync(asum, sizeof(double), cudaCpuDeviceId, NULL);

    printf("Absolute sum of vector size 2^%lu: %lf \n", N, *asum);

    // Verify result on CPU
    double err_asum = 0.0;
    for (size_t i = 0; i < ARRAY_SIZE; i++)
        err_asum += fabs(a[i]);

    if (fabs(err_asum - *asum) > 1e-2)
        printf("Error encountered: \n Function result: %lf \n Error checking result: %lf \n Error difference: %lf \n", *asum, err_asum, (err_asum - *asum));
    else
        printf("No errors encountered. \n (Difference less than 1e-2) \n Function result: %lf \n Error checking result: %lf \n Error difference: %lf \n", *asum, err_asum, (err_asum - *asum));

    cudaFree(a);
    cudaFree(asum);
    return 0;
}

Overwriting CUDA_asum6.cu


In [25]:
%%bash
nvcc CUDA_asum6.cu -lm -o CUDA_asum6 -Wno-deprecated-gpu-targets -arch=sm_60

In [26]:
%%bash
nvprof ./CUDA_asum6

==1009060== NVPROF is profiling process 1009060, command: ./CUDA_asum6


*** function = Double asum (GPU init data)
numElements = 268435456
numBlocks = 262144, numThreads = 1024 
Absolute sum of vector size 2^28: 108762865473.984177 
No errors encountered. 
 (Difference less than 1e-2) 
 Function result: 108762865473.984177 
 Error checking result: 108762865473.985641 
 Error difference: 0.001465 


==1009060== Profiling application: ./CUDA_asum6
==1009060== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   99.98%  18.4964s        30  616.55ms  614.59ms  641.18ms  asumfunc(unsigned long, double*, double*)
                    0.02%  3.6316ms         1  3.6316ms  3.6316ms  3.6316ms  init_array(unsigned long, double*)
      API calls:   85.35%  18.5016s        31  596.83ms  3.6929ms  641.25ms  cudaDeviceSynchronize
                    7.34%  1.59110s         4  397.78ms  591.59us  1.56934s  cudaMemPrefetchAsync
                    6.13%  1.32845s         2  664.23ms  1.1223ms  1.32733s  cudaMallocManaged
                    0.66%  143.21ms         2  71.604ms  1.2653ms  141.94ms  cudaFree
                    0.52%  112.34ms        31  3.6239ms  194.94us  98.757ms  cudaLaunchKernel
                    0.00%  1.0763ms       114  9.4410us     103ns  726.11us  cuDeviceGetAttribute
                    0.00%  211.62us    